<a href="https://colab.research.google.com/github/maxwiedmann/DSI_project_5__ee_water_levels/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### run in Google Colab

In [ ]:
'''
var geometry = 
    /* color: #ffc82d */
    /* displayProperties: [
      {
        "type": "rectangle"
      }
    ] */ 
    ee.Geometry.Polygon(
        [[[-120.16622848272344, 39.25150659029322],
          [-120.16622848272344, 38.93441896076937],
          [-119.91937942266485, 38.93441896076937],
          [-119.91937942266485, 39.25150659029322]]], null, false);
var landsat8= ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterBounds(geometry);
var waterThreshold = 0; 

// water function:
var waterfunction = function(image){
  //add the NDWI band to the image
  var ndwi = image.normalizedDifference(['B3', 'B5']).rename('NDWI');
  //get pixels above the threshold
  var water01 = ndwi.gt(waterThreshold);
  //mask those pixels from the image
  image = image.updateMask(water01).addBands(ndwi);

  var area = ee.Image.pixelArea();
  var waterArea = water01.multiply(area).rename('waterArea');

  image = image.addBands(waterArea);

  var stats = waterArea.reduceRegion({
    reducer: ee.Reducer.sum(), 
    geometry: geometry, 
    scale: 30,
  });

  return image.set(stats);
};
var collection = landsat8.map(waterfunction);
print(collection);

var chart = ui.Chart.image.series({
  imageCollection: collection.select('waterArea'), 
  region: geometry, 
  reducer: ee.Reducer.mean(), 
  scale: 30,
});
print(chart);


var landsat82 = ee.ImageCollection('LANDSAT/LC8_L1T_TOA').filterBounds(geometry);

var waterfunction2 = function(image){
  //add the NDWI band to the image
  var ndwi2 = image.normalizedDifference(['B3', 'B5']).rename('NDWI');
  var water02 = ndwi2.gt(waterThreshold+1);
  return image.updateMask(water02).addBands(ndwi2);
};

var col = landsat82.map(waterfunction2).select('NDWI');
var region = geometry;

col = col.map(function(img) {
  var doy = ee.Date(img.get('system:time_start')).getRelative('week', 'year');
  return img.set('doy', doy);
});

var distinctDOY = col.filterDate('2013-01-01', '2020-12-31');


var filter = ee.Filter.equals({leftField: 'doy', rightField: 'doy'});

var join = ee.Join.saveAll('doy_matches');

var joinCol = ee.ImageCollection(join.apply(distinctDOY, col, filter));

var comp = joinCol.map(function(img) {
  var doyCol = ee.ImageCollection.fromImages(
    img.get('doy_matches')
  );
  return doyCol.reduce(ee.Reducer.median());
});


var visParams = {
  palette: [
    'D4D0F6',	'CCC7F4',	'C3BEF3',	'BBB5F1',
'B1ACEF',	'A6A3EB',	'969AE6',	'8290DF',
'6F88D7',	'597DCC',	'4774C0',	'396BB1',
'3064A3',	'2A5C95',	'275186',	'27467A',
'283B6E',	'293265',	'2A295B',	'2B1E51',
]
};

var rgbVis = comp.map(function(img) {
  return img.visualize(visParams);
});

var gifParams = {
  'region': region,
  'dimensions': 300,
  'framesPerSecond': 12,
  'format': 'gif'
};

print(rgbVis.getVideoThumbURL(gifParams));

print(ui.Thumbnail(rgbVis, gifParams));
'''

# need to convert to Python

In [1]:
!earthengine authenticate 

Instructions for updating:
non-resource variables are not supported in the long term
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=VYkj8ZU3gexCOuIXQRCmU4ciRJcMCTxPy_s5cOl9018&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/2wGNQsaUAgw8oSLz3OqKRxTzxXC178ZDh6wB69hGh6yQWKhCrirlW4Y

Successfully saved authorization token.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import ee
ee.Initialize()

In [7]:
geometry =  ee.Geometry.Polygon( # Oroville Lake
        [[[-120.16622848272344, 39.25150659029322],
          [-120.16622848272344, 38.93441896076937],
          [-119.91937942266485, 38.93441896076937],
          [-119.91937942266485, 39.25150659029322]]], None, False);

landsat8= ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterBounds(geometry);



In [9]:
# adapted from: https://gis.stackexchange.com/questions/253164/counting-number-of-pixel-identified-as-water-from-a-collection-of-landsat-image

waterThreshold = 0

def waterfunction(image):
  #add the NDWI band to the image
  ndwi = image.normalizedDifference(['B3', 'B5']).rename('NDWI')
  #get pixels above the threshold
  water01 = ndwi.gt(waterThreshold)
  #mask those pixels from the image
  image = image.updateMask(water01).addBands(ndwi)

  area = ee.Image.pixelArea();
  waterArea = water01.multiply(area).rename('waterArea');

  image = image.addBands(waterArea);

  stats = waterArea.reduceRegion({
    reducer: ee.Reducer.sum(), 
    geometry: geometry, 
    scale: 30,
  })

  return image.set(stats);

In [ ]:

# Chart some interesting locations.
pt1 = ee.Geometry.Point(116.4647, 40.1054)

# ee.ImageCollection to ee.Image
img_cumulative = stackCollection(cumulative)


series = img_cumulative.reduceRegions(collection=pt1,
                                      reducer=ee.Reducer.mean(),
                                      scale=500)

dic_series = series.getInfo()
EVI_anom = np.array(list(dic_series['features'][0]['properties'].values()))

plt.plot(EVI_anom)
plt.show()

In [18]:
collection = landsat8.map(waterfunction)

#img_cumulative = stackCollection(collection)

series = img_cumulative.reduceRegions(collection = collection,
                                      reducer:ee.Reducer.mean(),
                                      scale = 30)

#dic_series = series.getInfo()
#print(dic_series)

NameError: ignored

In [11]:


collection = landsat8.map(waterfunction)

p

'''
chart = ui.Chart.image.series({
  imageCollection: collection.select('waterArea'), 
  region: geometry, 
  reducer: ee.Reducer.mean(), 
  scale: 30,
});
print(chart);
'''

NameError: ignored

In [ ]:
collection = landsat8.map(waterfunction);
print(collection);

chart = ui.Chart.image.series({
  imageCollection: collection.select('waterArea'), 
  region: geometry, 
  reducer: ee.Reducer.mean(), 
  scale: 30,
});
print(chart);


var landsat82 = ee.ImageCollection('LANDSAT/LC8_L1T_TOA').filterBounds(geometry);

var waterfunction2 = function(image){
  //add the NDWI band to the image
  var ndwi2 = image.normalizedDifference(['B3', 'B5']).rename('NDWI');
  var water02 = ndwi2.gt(waterThreshold+1);
  return image.updateMask(water02).addBands(ndwi2);
};

var col = landsat82.map(waterfunction2).select('NDWI');
var region = geometry;

col = col.map(function(img) {
  var doy = ee.Date(img.get('system:time_start')).getRelative('week', 'year');
  return img.set('doy', doy);
});

var distinctDOY = col.filterDate('2013-01-01', '2020-12-31');


var filter = ee.Filter.equals({leftField: 'doy', rightField: 'doy'});

var join = ee.Join.saveAll('doy_matches');

var joinCol = ee.ImageCollection(join.apply(distinctDOY, col, filter));

var comp = joinCol.map(function(img) {
  var doyCol = ee.ImageCollection.fromImages(
    img.get('doy_matches')
  );
  return doyCol.reduce(ee.Reducer.median());
});


var visParams = {
  palette: [
    'D4D0F6',	'CCC7F4',	'C3BEF3',	'BBB5F1',
'B1ACEF',	'A6A3EB',	'969AE6',	'8290DF',
'6F88D7',	'597DCC',	'4774C0',	'396BB1',
'3064A3',	'2A5C95',	'275186',	'27467A',
'283B6E',	'293265',	'2A295B',	'2B1E51',
]
};

var rgbVis = comp.map(function(img) {
  return img.visualize(visParams);
});

var gifParams = {
  'region': region,
  'dimensions': 300,
  'framesPerSecond': 12,
  'format': 'gif'
};

print(rgbVis.getVideoThumbURL(gifParams));

print(ui.Thumbnail(rgbVis, gifParams));
'''


---
### <i>Classifier Modeling</i>
----

In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier,\
AdaBoostClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,\
ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
#
import xgboost as xgb

----
Here we will import water–level depths from Google Earth Engine measured by counting isolated water pixels over time. In conjunction with the image reading we have obtained a csv file from NOAA (National Oceanic and Atmospheric Administration) detailing storm data in Butte County, CA. This data spans from approximately XXXXXXX to XXXXXXX.

Gaps in calendar dates between the two csv files will be filled with a `daterange()` function while missing water levels will be imputed with a seasonal mean in an effort to maintain our record counts. Alleviating missing values (i.e. images that were not recorded in GEE for certain dates) could concisderably affect our model if any gaps in months/seasons were observed.

----

In [ ]:
# Only for city of oroville???
water_levels = pd.read_csv('./oroville_water_levels.csv')

In [ ]:
# Find breakout percentage of flood occurrences for baseline
    # model and plot confusion matrix to optimize for type II errors (i.e no flood when true is flood)

# It is key that we stratify Y here as there are few instances of our target (floods) actually recorded. 
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.33,
                                                    stratify = y,
                                                    random_state = 42)

lr = LogisticRegression()
lr.fit(X_train_df, y_train)
n1 = '\n'
print(f"Our preliminary model will predict flood occurrence with {round((lr.score(X_train_df, y_train))*100,1)}% accurracy.{n1}Untrained data will predict with {round((lr.score(X_test_df, y_test))*100,1)}% accurracy.")

In [ ]:
# Ensemble Models
knn_pipe = Pipeline([('ss', StandardScaler(with_mean = False)),
                    ('knn', KNeighborsClassifier())])

vote = VotingClassifier([('ada', AdaBoostClassifier()),
                        ('grad_boost', GradientBoostingClassifier()),
                         ('XGB', xgb.XGBClassifier()),
                        ('knn_pipe', knn_pipe)])

# MUST USE DUNDERS TO DENOTE MODELS B/C OF SHARED PARAMS
params = {
    'ada__n_estimators': [100, 150, 200],
    'knn_pipe__knn__n_neighbors' : [1,2,3,4,5,10,20],
    'XGB__max_depth' : [10, 20, 30, 40],
    'XGB__booster' : ['gbtree', 'gblinear', 'dart']
}
# with KNN there are two levels to go through i.e 2 dunders
gs = GridSearchCV(vote, params, cv=5)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

In [ ]:
knn_pipe = Pipeline([('ss', StandardScaler(with_mean = False)),
                    ('knn', KNeighborsClassifier())])

vote = VotingClassifier([('ada', AdaBoostClassifier()),
                        ('grad_boost', GradientBoostingClassifier()),
                         ('XGB', xgb.XGBClassifier()),
                        ('knn_pipe', knn_pipe)])

# MUST USE DUNDERS TO DENOTE MODELS B/C OF SHARED PARAMS
params = {
    'ada__n_estimators': [100, 150, 200],
    'knn_pipe__knn__n_neighbors' : [1,2,3,4,5,10,20],
    'XGB__max_depth' : [10, 20, 30, 40],
    'XGB__booster' : ['gbtree', 'gblinear', 'dart']
}
# with KNN there are two levels to go through i.e 2 dunders
gs = GridSearchCV(vote, params, cv=5)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

---
#### Model Measurement
---

In [ ]:
preds = gs.predict(X_test)
tn,fp,fn,tp = confusion_matrix(y_test,preds).ravel()
plot_confusion_matrix(gs, X_test, y_test, cmap='Blues', values_format='d');

# Calc'ing how often it is accurately predicting flooding
tp/(tp+fn)